In [4]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import warnings
import numpy as np
import matplotlib.pyplot as plt
import os

warnings.filterwarnings('ignore') # suppress some MDAnalysis warnings about PSF files
print("Using MDAnalysis version", mda.__version__)


aminoacids = [
    "ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS", 
    "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"
]
results_folder = "resultats"
data_folder = "dades"
molecule_file = input("quin fitxer de molecules vols analitzar? ")
molecule_name = molecule_file.split(".")[0]

def folder_check(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

folder_check(results_folder)
folder_check(results_folder + "/" + molecule_name)

#funcions definides

def histogram(hist_data, hist_name, save):
    """
    prints and saves a histogram with the given distribution
    """
    hist_data = np.array(hist_data)
    
    fig, ax = plt.subplots()
    ax.hist(hist_data, bins=40 )
    # plot the xdata locations on the x axis:
    ax.plot(hist_data, 0*hist_data, 'd' )
    ax.set_ylabel('Nombre de residuus amb aquesta RMSD')
    ax.set_xlabel('RMSD '+ hist_name)
    if save == True:
        plt.savefig(results_folder + "/" + molecule_name+ "/"+'histogram_'+hist_name+'.png', dpi=1000, bbox_inches='tight')


def average_structure(av_residu, av_univers, av_save):
    """
    For a given aminoacid and a universe, returns the average shape of that aminoacid in the universe
    """
    av_univers = av_univers.select_atoms("protein and resname "+av_residu)
    n_frames = len(av_univers.residues)
    if n_frames ==0:
        return
    n_atoms = len(av_univers.residues[0].atoms)
    with mda.Writer(results_folder + "/" + molecule_name+ "/" +av_residu +'.pdb', n_atoms) as w:
        for ts in range(n_frames):
            if len(av_univers.residues[ts].atoms)== n_atoms:
                w.write(av_univers.residues[ts].atoms)
            else: print(ts, "té algun problema a la " + av_residu)
    av_univers=mda.Universe(results_folder + "/" + molecule_name+ "/"+av_residu +'.pdb')
    average = align.AverageStructure(av_univers,
                                     ref_frame=0).run()
    ref = average.results.universe
    ref.atoms.write(results_folder + "/" + molecule_name+ "/"+"average_shape_" + av_residu+ " .pdb")
    
    mda.analysis.align.AlignTraj(av_univers, ref, select="protein", filename = results_folder + "/" + molecule_name+ "/" +"aligned_sequence_"+av_residu+".pdb").run()
    dist_dist = mda.analysis.rms.RMSD(av_univers, ref)
    dist_dist.run()
    
    histogram([av_result[2] for av_result in dist_dist.results.rmsd], av_residu, av_save)
    os.remove(results_folder + "/" + molecule_name+ "/" +av_residu +'.pdb')

#Main
u = mda.Universe(data_folder+"/"+molecule_file)
#Es determinen les estructures mitjanes
save_histograms= False
for aminoacid in aminoacids:
    average_structure(aminoacid, u, save_histograms)



Using MDAnalysis version 2.8.0


quin fitxer de molecules vols analitzar?  fold_9mrf_prediction_model_0.cif


ValueError: 'CIF' isn't a valid topology format, nor a coordinate format
   from which a topology can be minimally inferred.
   You can use 'Universe(topology, ..., topology_format=FORMAT)'
   to explicitly specify the format and
   override automatic detection. Known FORMATs are:
   dict_keys(['PSF', 'TOP', 'PRMTOP', 'PARM7', 'PDB', 'ENT', 'XPDB', 'PQR', 'GRO', 'CRD', 'PDBQT', 'DMS', 'TPR', 'MOL2', 'DATA', 'LAMMPSDUMP', 'XYZ', 'TXYZ', 'ARC', 'GMS', 'CONFIG', 'HISTORY', 'XML', 'MMTF', 'GSD', 'MINIMAL', 'ITP', 'IN', 'FHIAIMS', 'PARMED', 'RDKIT', 'OPENMMTOPOLOGY', 'OPENMMAPP'])
   See https://docs.mdanalysis.org/documentation_pages/topology/init.html#supported-topology-formats
   For missing formats, raise an issue at 
   https://github.com/MDAnalysis/mdanalysis/issues

In [6]:
mda.analysis.align.AlignTraj??

Init signature:
mda.analysis.align.AlignTraj(
    mobile,
    reference,
    select='all',
    filename=None,
    prefix='rmsfit_',
    weights=None,
    tol_mass=0.1,
    match_atoms=True,
    strict=False,
    force=True,
    in_memory=False,
    writer_kwargs=None,
    **kwargs,
)
Source:        
class AlignTraj(AnalysisBase):
    """RMS-align trajectory to a reference structure using a selection.

    Both the reference `reference` and the trajectory `mobile` must be
    :class:`MDAnalysis.Universe` instances. If they contain a trajectory then
    it is used. The output file format is determined by the file extension of
    `filename`. One can also use the same universe if one wants to fit to the
    current frame.

    .. versionchanged:: 1.0.0
       ``save()`` has now been removed, as an alternative use ``np.savetxt()``
       on :attr:`results.rmsd`.

    """

    def __init__(self, mobile, reference, select='all', filename=None,
                 prefix='rmsfit_', weights=None,

In [5]:
from ase.io import read, write

# Load the CIF file
atoms = read("fold_9mrf_prediction_model_0.cif")

# Write to PDB format
write("fold_9mrf_prediction_model_0.pdb", atoms)

print("Conversion completed: 'output_file.pdb' created.")

ModuleNotFoundError: No module named 'ase'

In [41]:
#Funcions definides que no han resultat útils al final
def distortion_distribution(dist_univers, dist_residu, dist_reference):
    """
    for a given aminoacid it calclates the RMSD 
    """
    dist_univers = dist_univers.select_atoms("resname "+dist_residu)
    distribution= []
    for res_i in dist_univers.residues:
        distribution.append(mda.analysis.rms.RMSD(dist_reference, res_i.atoms))
    histogram(distribution, dist_residu)
    print(len(distribution))
    

def max_list(m_list, m_select):
    """
    Given a list of tuples it returns the one that has the max value at a given position
    """
    m_max= m_list[0]
    for element in m_list:
        if element[m_select]>m_max[m_select]:
            m_max = element
    return m_max

def n_alpha_dist(protein):
    """
    For a given protein, it returns the distance between the c_alpha atom and its -COO atached carbon for each residue
    """
    if not isinstance(protein, mda.core.universe.Universe):
        raise TypeError(f"Expected input_value to be of type mda.core.universe.Universe, but got {type(protein).__name__}")
    distances = []
    for res in protein.residues:
        nitrogen = res.atoms.select_atoms("name N")
        carbon_a = res.atoms.select_atoms("name CA")
        if len(nitrogen) == 1 and len(carbon_a) == 1:
            distance = np.linalg.norm(nitrogen.positions[0] - carbon_a.positions[0])
            distances.append((res.resid, res.resname, distance))
        else:
            print(f"Skipping residue {res.resid} ({res.resname}) due to lacking infotmation")
    return distances
